<a href="https://colab.research.google.com/github/ArunK-ML/Project---Nutrition-Paradox-A-Global-View-on-Obesity-and-Malnutrition/blob/main/Nutrition_Paradox_Streamlit_Final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Streamlit**

In [ ]:
%%writefile streamlit_app.py
# streamlit_app.py
import streamlit as st
import sqlite3
import pandas as pd
import plotly.express as px

st.set_page_config(layout="wide")

# Connect to SQLite DB
def get_db_connection():
    return sqlite3.connect("health_database.db")

# Execute SQL Query
def run_query(query):
    conn = get_db_connection()
    try:
        df = pd.read_sql(query, conn)
        return df
    except Exception as e:
        st.error(f"Error executing query: {e}")
        return None
    finally:
        conn.close()

# Sidebar Navigation
st.sidebar.title("🌠 Nutrition Dashboard Menu")
page = st.sidebar.radio("Navigate", ["Home", "Filter Criteria", "Queries", "About"])

# Query Lists
Obesity_Table_questions = [
    "1. Top 5 regions with the highest average obesity levels in the most recent year(2022)",
    "2. Top 5 countries with highest obesity estimates",
    "3. Obesity trend in India over the years(Mean_estimate)",
    "4. Average obesity by gender",
    "5. Country count by obesity level category and age group",
    "6. Top 5 countries least reliable countries(with highest CI_Width) and Top 5 most consistent countries (smallest average CI_Width)",
    "7. Average obesity by age group",
    "8. Top 10 Countries with consistent low obesity (low average + low CI)over the years",
    "9. Countries where female obesity exceeds male by large margin (same year)",
    "10. Global average obesity percentage per year"
]

Obesity_queries = [
      "SELECT Region, AVG(Mean_Estimate) AS avg_obesity FROM obesity WHERE Year = 2022 GROUP BY Region ORDER BY avg_obesity DESC LIMIT 5;",
      "SELECT Country, MAX(Mean_Estimate) AS max_obesity FROM obesity GROUP BY Country ORDER BY max_obesity DESC LIMIT 5;",
      "SELECT Year, Mean_Estimate FROM obesity WHERE Country = 'India' ORDER BY Year;",
      "SELECT Gender, AVG(Mean_Estimate) AS avg_obesity FROM obesity GROUP BY Gender;",
      "SELECT Country, age_group, COUNT(*) AS country_count FROM obesity GROUP BY Country, age_group;",
      "SELECT Country, AVG(CI_Width) AS avg_ci_width FROM obesityGROUP BY Country ORDER BY avg_ci_width ASC LIMIT 5;",
      "SELECT Country, AVG(CI_Width) AS avg_CI_Width FROM obesity GROUP BY Country ORDER BY avg_CI_Width ASC LIMIT 5;",
      "SELECT age_group, AVG(Mean_Estimate) AS avg_obesity FROM obesity GROUP BY age_group;",
      "SELECT Country, AVG(Mean_Estimate) AS avg_obesity, AVG(CI_Width) AS avg_ci_width FROM obesity GROUP BY Country ORDER BY avg_obesity;",
      "SELECT Year, Country, Mean_Estimate FROM obesity WHERE Gender = 'Female' AND Mean_Estimate > (SELECT Mean_Estimate FROM obesity WHERE Gender = 'Male' AND Year = obesity.Year)ORDER BY Year, Mean_Estimate DESC;",
      ]

my_queries = [
    "16. Find asteroids with an average speed > 40,000 km/h",
    "17. Calculate average miss distance for each asteroid",
    "18. List top 5 largest potentially hazardous asteroids",
    "19. Find asteroids that approached Earth exactly once",
    "20. Average size (diameter) of hazardous vs. non-hazardous asteroids",
    "21. Asteroids with multiple approaches on the same day",
    "22. All asteroids that have never been hazardous",
    "23. Day with most asteroid approaches",
    "24. Asteroids that only approached Earth in 2025",
    "25. Asteroid with most total approaches"
]

my_queries_list = [
    "SELECT a.name, AVG(ca.relative_velocity_kmph) AS avg_velocity FROM asteroids a JOIN close_approach ca ON a.id = ca.neo_reference_id GROUP BY a.id HAVING avg_velocity > 40000;",
    "SELECT neo_reference_id, AVG(miss_distance_km) AS avg_miss_distance_km FROM close_approach GROUP BY neo_reference_id;",
    "SELECT name, estimated_dia_min_km FROM asteroids WHERE is_potentially_hazardous = 1 ORDER BY estimated_dia_min_km DESC LIMIT 5;",
    "SELECT a.name FROM asteroids a JOIN close_approach ca ON a.id = ca.neo_reference_id GROUP BY a.id HAVING COUNT(*) = 1;",
    "SELECT is_potentially_hazardous, AVG((estimated_dia_min_km + estimated_dia_max_km) / 2) AS avg_diameter_km FROM asteroids GROUP BY is_potentially_hazardous;",
    "SELECT neo_reference_id, close_approach_date, COUNT(*) AS approaches FROM close_approach GROUP BY neo_reference_id, close_approach_date HAVING approaches > 1;",
    "SELECT id, name FROM asteroids WHERE is_potentially_hazardous = 0;",
    "SELECT close_approach_date, COUNT(*) AS total_approaches FROM close_approach GROUP BY close_approach_date ORDER BY total_approaches DESC LIMIT 1;",
    "SELECT DISTINCT a.name FROM asteroids a JOIN close_approach ca ON a.id = ca.neo_reference_id WHERE strftime('%Y', ca.close_approach_date) = '2025';",
    "SELECT neo_reference_id, COUNT(*) AS total_approaches FROM close_approach GROUP BY neo_reference_id ORDER BY total_approaches DESC LIMIT 1;"
]

# === Page Logic ===
st.title("☄️ Asteroid Insights Explorer")

if page == "Home":
    st.header("Welcome to the Asteroid Data Dashboard")

    col1, col2 = st.columns([2,2])  # Wider text column on the left

    with col1:
        st.markdown("""
        ### 🚀 Explore NASA's Near-Earth Object Data
        This dashboard allows you to:
        - View asteroids' close approaches
        - Explore their speeds, sizes, and distances
        - Check which are potentially hazardous
        - Download filtered results for further analysis

        Navigate using the sidebar to run queries or apply custom filters.
        """)

    with col2:
        st.image("/content/170275-843069693_tiny.jpg")

elif page == "Filter Criteria":
    st.header("Filter Criteria")
    st.info("Use the filter options below to explore specific asteroid characteristics.")

    col1, col2, col3 = st.columns(3)
    with col1:
        sl_min_magnitude = st.slider("Min Magnitude", 13.80, 32.61, (13.80, 20.34))
    with col2:
        sl_relative_velocity = st.slider("Relative Velocity (km/h)", 1418.21, 173071.83, (1600.00, 50000.00))
    with col3:
        start_date = st.date_input("Start Date")

    col4, col5, col6 = st.columns(3)
    with col4:
        sl_min_estimated_dia_range = st.slider("Min Estimated Diameter (km)", 0.00, 4.62, (1.0, 2.10))
    with col5:
        sl_max_estimated_dia_range = st.slider("Max Estimated Diameter (km)", 0.00, 10.33, (1.0, 6.0))
    with col6:
        end_date = st.date_input("End Date")

    col7, col8 = st.columns(2)
    with col7:
        sl_astronomical_unit = st.slider("Astronomical Unit", 0.00, 0.58, (0.05, 0.48))
    with col8:
        sl_potentially_hazardous = st.selectbox("Is Potentially Hazardous", [0, 1])

    if st.button("Filter"):
        query = f"""
        SELECT a.name, ca.close_approach_date, ca.relative_velocity_kmph, ca.astronomical,
               ca.miss_distance_km, a.absolute_magnitude_h, a.estimated_dia_max_km, a.is_potentially_hazardous
        FROM close_approach ca
        JOIN asteroids a ON a.id = ca.neo_reference_id
        WHERE a.absolute_magnitude_h BETWEEN {sl_min_magnitude[0]} AND {sl_min_magnitude[1]}
          AND a.estimated_dia_min_km BETWEEN {sl_min_estimated_dia_range[0]} AND {sl_min_estimated_dia_range[1]}
          AND a.estimated_dia_max_km BETWEEN {sl_max_estimated_dia_range[0]} AND {sl_max_estimated_dia_range[1]}
          AND ca.relative_velocity_kmph BETWEEN {sl_relative_velocity[0]} AND {sl_relative_velocity[1]}
          AND ca.astronomical BETWEEN {sl_astronomical_unit[0]} AND {sl_astronomical_unit[1]}
          AND a.is_potentially_hazardous = {sl_potentially_hazardous}
          AND date(ca.close_approach_date) BETWEEN '{start_date}' AND '{end_date}'
        """
        df = run_query(query)
        if df is not None and not df.empty:
            st.dataframe(df, use_container_width=True)
            fig = px.histogram(df, x="relative_velocity_kmph", title="Velocity Distribution")
            st.plotly_chart(fig, use_container_width=True)
            st.download_button("Download CSV", df.to_csv(index=False), "filtered_results.csv")
        else:
            st.warning("No matching data found for selected criteria.")

elif page == "Queries":
    st.header("Predefined SQL Queries")
    category = st.radio("Choose Query Type", ["Guvi Queries", "My Queries"])
    if category == "Guvi Queries":
        desc = st.selectbox("Select Query:", guvi_questions)
        query = guvi_queries[guvi_questions.index(desc)]
    else:
        desc = st.selectbox("Select Query:", my_queries)
        query = my_queries_list[my_queries.index(desc)]

    if st.button("Run Query"):
        df = run_query(query)
        if df is not None and not df.empty:
            st.subheader(desc)
            st.dataframe(df, use_container_width=True)
            numeric_cols = df.select_dtypes(include=["int64", "float64"]).columns
            if len(numeric_cols) > 0:
                col = st.selectbox("Select column to plot", numeric_cols)
                fig = px.bar(df, x=df.columns[0], y=col, title="Visual Representation")
                st.plotly_chart(fig, use_container_width=True)
        else:
            st.warning("No data found or query failed.")

elif page == "About":
    st.header("About This App")
    st.write("Built using Streamlit, this app allows exploration of near-Earth asteroid data with SQL and interactive charts.")
    st.write("Data Collected through Nasa API Link")
    st.write("**Project Given by Guvi**")
    st.write("**Thank you for visit**")


st.markdown("---")
st.caption("Developed by Arun Kumar | Powered by NASA NEO Data")
